In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version="2025-01-01-preview",
    azure_endpoint="https://nam-m9jcglr5-eastus2.cognitiveservices.azure.com/",
    api_key="Be4KiOwnk918zadWvT8VeVoYq3SFNNzCh4WrntFHoVubgr3bRdPqJQQJ99BDACHYHv6XJ3w3AAAAACOG7mDD"
)

In [3]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_completion_tokens=800,
    temperature=1.0,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    model="gpt-4.1-nano"
)

In [4]:
print(response.choices[0].message.content)

That sounds like an exciting trip! Paris is full of incredible sights and experiences. Here are some must-see attractions and activities:

1. Eiffel Tower: No trip to Paris is complete without visiting this iconic landmark. Consider going up to the top for a panoramic view of the city.
2. Louvre Museum: Home to thousands of works of art, including the Mona Lisa and the Venus de Milo. It’s one of the world’s largest and most visited art museums.
3. Notre-Dame Cathedral: Though it was damaged by fire in 2019, the cathedral remains a stunning example of Gothic architecture. Visit the nearby Île de la Cité.
4. Sacré-Cœur Basilica: Located at the highest point in the city in Montmartre, it offers spectacular views and a charming neighborhood to explore.
5. Musée d'Orsay: An art museum housed in a former railway station, featuring masterpieces from Impressionist and Post-Impressionist artists.
6. Champs-Élysées & Arc de Triomphe: Stroll along this famous avenue and visit the monumental Arc d

## prepare data batch

In [7]:
import os
import pandas as pd
import threading

# Get all CSV files in data_version1 directory
csv_files = []
excluded_files = []

for file in os.listdir('../data/data_version1/'):
    if file.endswith('.csv') and file.startswith('reason_specialist'):
        if not any(excluded in file for excluded in excluded_files):
            csv_files.append(file)

print(f"Found {len(csv_files)} CSV files to process: {csv_files}")

# Process each file in a separate thread
def process_all_files():
    threads = []
    
    # Create an empty list to store all dataframes
    all_dfs = []
    
    for i, file in enumerate(csv_files):
        # Read the file
        file_path = f'../data/data_version1/{file}'
        df = pd.read_csv(file_path, index_col=0)
        df = df.reset_index(drop=True)
        
        # Add the dataframe to the list
        all_dfs.append(df)
    
    # Merge all dataframes into one
    merged_df = pd.concat(all_dfs, ignore_index=True)
    
    return merged_df


# Start processing
df = process_all_files()
print("All files merged successfully")


Found 18 CSV files to process: ['reason_specialist - thần kinh.csv', 'reason_specialist - vô sinh hiếm muộn.csv', 'reason_specialist - nhi khoa.csv', 'reason_specialist - thận tiết niệu.csv', 'reason_specialist - ung bướu.csv', 'reason_specialist - hô hấp phổi.csv', 'reason_specialist - chuyên khoa mắt.csv', 'reason_specialist - cơ xương khớp.csv', 'reason_specialist - nha khoa.csv', 'reason_specialist - tim mạch.csv', 'reason_specialist - tiêu hóa.csv', 'reason_specialist - sức khỏe tâm thần.csv', 'reason_specialist - nội khoa.csv', 'reason_specialist - tiểu đường nội tiết.csv', 'reason_specialist - tai mũi họng.csv', 'reason_specialist - nam học.csv', 'reason_specialist - da liễu.csv', 'reason_specialist - sản phụ khoa.csv']
All files merged successfully


In [8]:
df.head()

,partner_id,specialist_id,status,gender,province_id,age,reason_combind,specialist_name,correct_prediction,ids,logs
0,2,18,2,NaN,1,57,mất ngủ,thần kinh,NaN,NaN,NaN
1,4,18,2,NaN,1,35,rối loạn thần kinh thực vật,thần kinh,NaN,NaN,NaN
2,4,18,2,NaN,11,36,đau đầu,thần kinh,NaN,NaN,NaN
3,4,18,2,NaN,1,40,"đau đầu,đau sau ngực gần phổi",thần kinh,NaN,NaN,NaN
4,3,18,2,NaN,1,12,co giật 3 lần,thần kinh,NaN,NaN,NaN


In [9]:
df.to_csv('../data/data_version3/dataset.csv', index=False)

In [10]:
system_prompt = """You are a system that converts examination reasons into symptoms and diseases ONLY when they actually exist.

# Conversion rules:
1. Only convert when actual medical conditions are present
2. Correct spelling errors before formatting
3. Format as: "symptom1, symptom2, disease1, disease2"
4. Return null when:
   - Symptoms/diseases don't match the specialist field (chuyên khoa)
   - No current symptoms exist (except for "nội khoa" specialty)
   - Past surgeries without current symptoms ("đã mổ...")
   - Medical history without current symptoms ("tiền sử...")
   - Information that isn't a symptom/disease
5. Special case: For "nội khoa" (internal medicine), accept "khám tổng quát" (general checkup) as is

# Return null for:
- Tiền sử phẫu thuật không có triệu chứng (Surgical history without symptoms)
- Khám định kỳ (routine checkups) (except "nội khoa")
- Tái khám (follow-up appointments)
- Yêu cầu hành chính (administrative requests)
- Thủ thuật quá khứ không có triệu chứng (past procedures without symptoms)
- Yêu cầu hồ sơ y tế (medical record requests)
- Kết quả xét nghiệm không có triệu chứng (test results without symptoms)

# Examples:
1. Input: "Đau đầu, chóng mặt, buồn nôn" (Specialty: Thần kinh)
   Output: {"result": "đau đầu, chóng mặt, buồn nôn"}

2. Input: "Xuất huyết dạ dày, đau bung" (Specialty: Tiêu hóa)
   Output: {"result": "xuất huyết dạ dày, đau bụng"}

3. Input: "Khám tổng quát" (Specialty: Nội khoa)
   Output: {"result": "khám tổng quát"}

4. Input: "Đã mổ ruột thừa 2 năm trước" (Specialty: Ngoại khoa)
   Output: {"result": null}

5. Input: "Khó thở, nghi ngờ hen xuyễn" (Specialty: Hô hấp)
   Output: {"result": "khó thở, hen suyễn"}

6. Input: "Xin giấy chứng nhận sức khỏe" (Specialty: Nội khoa)
   Output: {"result": null}

7. Input: "Ho, sốt cao, viêm phổi" (Specialty: Nhi)
   Output: {"result": "ho, sốt cao, viêm phổi"}

# Response:
{
    "result": "Processed symptoms/diseases or null"
}"""

In [11]:
user_prompt = "Reason: {reason}\nSpecialist: {specialist}"

In [12]:
print(user_prompt.format(reason="dau dau", specialist="than kinh"))

Reason: dau dau
Specialist: than kinh


In [27]:
import json

# Define the output file path
output_file = '../data/data_version3/dataset.jsonl'

# Process DataFrame and create JSONL entries
with open(output_file, 'w', encoding='utf-8') as f:
    for idx, row in df.iterrows():
        # Format the user prompt with data from the row
        formatted_user_prompt = user_prompt.format(
            reason=row.get('reason_combind', ''),
            specialist=row.get('specialist_name', '')
        )
        
        # Create a JSON object for each row
        json_entry = {
            "custom_id": f"task-{idx}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": [
                    {
                        "role": "system",
                        "content": system_prompt
                    },
                    {
                        "role": "user",
                        "content": formatted_user_prompt
                    }
                ]
            }
        }
        
        # Write the JSON object to the file
        f.write(json.dumps(json_entry) + '\n')

print(f"JSONL file created successfully at {output_file}")


JSONL file created successfully at ../data/data_version3/dataset.jsonl


In [13]:
df.head()

,partner_id,specialist_id,status,gender,province_id,age,reason_combind,specialist_name,correct_prediction,ids,logs
0,2,18,2,NaN,1,57,mất ngủ,thần kinh,NaN,NaN,NaN
1,4,18,2,NaN,1,35,rối loạn thần kinh thực vật,thần kinh,NaN,NaN,NaN
2,4,18,2,NaN,11,36,đau đầu,thần kinh,NaN,NaN,NaN
3,4,18,2,NaN,1,40,"đau đầu,đau sau ngực gần phổi",thần kinh,NaN,NaN,NaN
4,3,18,2,NaN,1,12,co giật 3 lần,thần kinh,NaN,NaN,NaN


In [2]:
import pandas as pd

df = pd.read_csv('../data/data_version3/dataset.csv')

In [3]:
df.head()

,partner_id,specialist_id,status,gender,province_id,age,reason_combind,specialist_name,correct_prediction,ids,logs
0,2,18,2,NaN,1,57,mất ngủ,thần kinh,NaN,NaN,NaN
1,4,18,2,NaN,1,35,rối loạn thần kinh thực vật,thần kinh,NaN,NaN,NaN
2,4,18,2,NaN,11,36,đau đầu,thần kinh,NaN,NaN,NaN
3,4,18,2,NaN,1,40,"đau đầu,đau sau ngực gần phổi",thần kinh,NaN,NaN,NaN
4,3,18,2,NaN,1,12,co giật 3 lần,thần kinh,NaN,NaN,NaN


In [5]:
system_prompt = """
Bạn là hệ thống xử lý văn bản y tế chuyển đổi lý do khám bệnh thành triệu chứng và bệnh tật bằng tiếng Việt.

# Định dạng đầu vào
{
  "records": [
    {
      "reason_combined": "Lý do khám bệnh của bệnh nhân",
      "specialist_name": "Chuyên khoa được tư vấn"
    },
    {...}
  ]
}

# Quy tắc cơ bản
1. Chỉ trích xuất các tình trạng y tế THỰC SỰ đang hiện diện
2. Sửa lỗi chính tả (ví dụ: "bung" → "bụng", "xuyễn" → "suyễn")
3. Định dạng như danh sách phân cách bằng dấu phẩy CHỈ BẰNG TIẾNG VIỆT: "triệu chứng1, triệu chứng2, bệnh1, bệnh2"
4. Trả về null khi không đạt điều kiện trích xuất
5. LUÔN đầu ra bằng tiếng Việt, không bao giờ dịch sang tiếng Anh hoặc ngôn ngữ khác

# Khi nào trả về null
- Triệu chứng/bệnh tật không phù hợp với chuyên khoa
- Không có triệu chứng hiện tại được đề cập (ngoại trừ chuyên khoa "nội khoa")
- Các sự kiện y tế trong quá khứ không có triệu chứng hiện tại ("đã mổ...", "tiền sử...")
- Yêu cầu hành chính, tái khám định kỳ, hoặc yêu cầu hồ sơ
- Kết quả xét nghiệm không có triệu chứng liên quan

# Trường hợp đặc biệt
- Chỉ đối với "nội khoa": "khám tổng quát" là hợp lệ và được giữ nguyên
- Không bao giờ thay đổi ý nghĩa y tế - chỉ sửa lỗi chính tả nhưng giữ nguyên tình trạng

# Định dạng phản hồi
Chỉ cung cấp đối tượng JSON hợp lệ với cấu trúc này:
{
  "results": [
    {"result": "văn bản trích xuất hoặc null"},
    {"result": "văn bản trích xuất hoặc null"},
    ...
  ]
}

# Ví dụ đầu vào/đầu ra
Đầu vào:
{
  "records": [
    {"reason_combined": "Đau đầu, chóng mặt, buồn nôn", "specialist_name": "Thần kinh"},
    {"reason_combined": "Xuất huyết dạ dày, đau bung", "specialist_name": "Tiêu hóa"},
    {"reason_combined": "Khám tổng quát", "specialist_name": "Nội khoa"},
    {"reason_combined": "Đã mổ ruột thừa 2 năm trước", "specialist_name": "Ngoại khoa"}
  ]
https://nam-m9jrkzng-swedencentral.openai.azure.com}

Đầu ra:
{
  "results": [
    {"result": "đau đầu, chóng mặt, buồn nôn"},
    {"result": "xuất huyết dạ dày, đau bụng"},
    {"result": "khám tổng quát"},
    {"result": null}
  ]
}
"""

In [7]:
import json
import os
import pandas as pd
from tqdm.notebook import tqdm
from openai import OpenAI
from together import Together

# model="hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"
# client = OpenAI(
#     api_key="UNKOWN",
#     base_url="https://ec32-34-138-204-180.ngrok-free.app/v1"
# )

system_prompt = """
Bạn là hệ thống xử lý văn bản y tế chuyển đổi lý do khám bệnh thành triệu chứng và bệnh tật bằng tiếng Việt.

# Định dạng đầu vào
{
  "records": [
    {
      "reason_combined": "Lý do khám bệnh của bệnh nhân",
      "specialist_name": "Chuyên khoa được tư vấn"
    },
    {...}
  ]
}

# Quy tắc cơ bản
1. Chỉ trích xuất các tình trạng y tế THỰC SỰ đang hiện diện
2. Sửa lỗi chính tả (ví dụ: "bung" → "bụng", "xuyễn" → "suyễn")
3. Định dạng như danh sách phân cách bằng dấu phẩy CHỈ BẰNG TIẾNG VIỆT: "triệu chứng1, triệu chứng2, bệnh1, bệnh2"
4. Trả về null khi không đạt điều kiện trích xuất
5. LUÔN đầu ra bằng tiếng Việt, không bao giờ dịch sang tiếng Anh hoặc ngôn ngữ khác

# Khi nào trả về null
- Triệu chứng/bệnh tật không phù hợp với chuyên khoa
- Không có triệu chứng hiện tại được đề cập (ngoại trừ chuyên khoa "nội khoa")
- Các sự kiện y tế trong quá khứ không có triệu chứng hiện tại ("đã mổ...", "tiền sử...")
- Yêu cầu hành chính, tái khám định kỳ, hoặc yêu cầu hồ sơ
- Kết quả xét nghiệm không có triệu chứng liên quan

# Trường hợp đặc biệt
- Chỉ đối với "nội khoa": "khám tổng quát" là hợp lệ và được giữ nguyên
- Không bao giờ thay đổi ý nghĩa y tế - chỉ sửa lỗi chính tả nhưng giữ nguyên tình trạng

# Định dạng phản hồi
Chỉ cung cấp đối tượng JSON hợp lệ với cấu trúc này:
{
  "results": [
    {"result": "văn bản trích xuất hoặc null"},
    {"result": "văn bản trích xuất hoặc null"},
    ...
  ]
}

# Ví dụ đầu vào/đầu ra
Đầu vào:
{
  "records": [
    {"reason_combined": "Đau đầu, chóng mặt, buồn nôn", "specialist_name": "Thần kinh"},
    {"reason_combined": "Xuất huyết dạ dày, đau bung", "specialist_name": "Tiêu hóa"},
    {"reason_combined": "Khám tổng quát", "specialist_name": "Nội khoa"},
    {"reason_combined": "Đã mổ ruột thừa 2 năm trước", "specialist_name": "Ngoại khoa"}
  ]
https://nam-m9jrkzng-swedencentral.openai.azure.com}

Đầu ra:
{
  "results": [
    {"result": "đau đầu, chóng mặt, buồn nôn"},
    {"result": "xuất huyết dạ dày, đau bụng"},
    {"result": "khám tổng quát"},
    {"result": null}
  ]
}
"""


client = Together(
    api_key="95981b63ee8bc37a47e013a2946aecab679285179eb3a3c24e1179e38e75c434"
)
model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

def batch_process_records(dataframe, batch_size=100, output_path='../data/data_version3/dataset_processed.csv'):
    """
    Process the dataframe in batches and update with results after each batch.
    Skips already processed records if the output file exists.
    """
    # Initialize or load processed dataframe
    if os.path.exists(output_path):
        processed_df = pd.read_csv(output_path)
        print(f"Found existing processed file with {len(processed_df)} records")
        
        # Check if we need to expand the processed dataframe (if input is larger)
        if len(dataframe) > len(processed_df):
            print(f"Input dataframe has {len(dataframe)} records, adding {len(dataframe) - len(processed_df)} new records")
            # Add new rows from dataframe that aren't in processed_df
            processed_df = pd.concat([processed_df, dataframe.iloc[len(processed_df):].copy()], ignore_index=False)
    else:
        processed_df = dataframe.copy()
        if 'processed_symptoms' not in processed_df.columns:
            processed_df['processed_symptoms'] = None
        print(f"Created new processed dataframe with {len(processed_df)} records")
    
    # Count records that need processing
    records_to_process = processed_df['processed_symptoms'].isna().sum()
    print(f"Found {records_to_process} records that need processing")
    
    if records_to_process == 0:
        print("All records are already processed. Nothing to do.")
        return processed_df
    
    total_records = len(processed_df)
    
    # Process only unprocessed records (where processed_symptoms is None/NaN)
    unprocessed_indices = processed_df[processed_df['processed_symptoms'].isna()].index.tolist()
    
    for batch_start in tqdm(range(0, len(unprocessed_indices), batch_size), desc="Processing data..."):
        batch_indices = unprocessed_indices[batch_start:batch_start + batch_size]
        
        print(f"Processing batch {batch_start//batch_size + 1}, records {batch_indices[0]} to {batch_indices[-1]}")
        
        # Extract batch records using the indices
        batch_df = processed_df.loc[batch_indices].copy()
        
        # Create input structure for the batch
        batch_records = []
        for _, row in batch_df.iterrows():
            batch_records.append({
                "reason_combined": row.get('reason_combind', ''),
                "specialist_name": row.get('specialist_name', '')
            })
        
        input_json = {"records": batch_records}
        print(f"Processing {len(batch_records)} records")
        
        # Process batch with Together API
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": "Input: " + json.dumps(input_json)}
            ],
            temperature=0.0,
            max_tokens=1024,
            response_format={"type": "json_object"},
        )
        
        # Parse the response
        try:
            results_text = response.choices[0].message.content
            # Handle both JSON and string responses
            try:
                results_data = json.loads(results_text)
                print(results_data)
                if "results" in results_data:
                    batch_results = results_data["results"]
                    # Update the processed_symptoms column for this batch
                    for i, result in enumerate(batch_results):
                        if i < len(batch_indices):
                            index = batch_indices[i]
                            processed_df.at[index, 'processed_symptoms'] = result.get('result')
                else:
                    print(f"Warning: Unexpected response format in batch {batch_start//batch_size + 1}")
            except json.JSONDecodeError:
                print(f"Warning: Response is not valid JSON in batch {batch_start//batch_size + 1}")
                lines = results_text.strip().split('\n')
                for i, line in enumerate(lines):
                    if i < len(batch_indices):
                        index = batch_indices[i]
                        processed_df.at[index, 'processed_symptoms'] = line if line.strip() else None
        except Exception as e:
            print(f"Error processing response in batch {batch_start//batch_size + 1}: {str(e)}")
        
        # Save the dataframe after each batch
        processed_df.to_csv(output_path, index=False)
        print(f"Progress saved after batch {batch_start//batch_size + 1}")
    
    print(f"Processing complete. Processed {records_to_process} out of {total_records} records.")
    return processed_df

# Process the dataset in batches
batch_size = 10
processed_df = batch_process_records(df, batch_size)

# Examine a few results
print("\nSample results:")
for i in range(min(5, len(processed_df))):
    print(f"Record {i+1}:")
    print(f"Reason: {processed_df.iloc[i]['reason_combind']}")
    print(f"Specialist: {processed_df.iloc[i]['specialist_name']}")
    print(f"Processed Symptoms: {processed_df.iloc[i]['processed_symptoms']}")
    print()

Created new processed dataframe with 53624 records
Found 53624 records that need processing


Processing data...:   0%|          | 0/5363 [00:00<?, ?it/s]

Processing batch 1, records 0 to 9
Processing 10 records
{'results': [{'result': 'mất ngủ'}, {'result': 'rối loạn thần kinh thực vật'}, {'result': 'đau đầu'}, {'result': 'đau đầu, đau sau ngực gốc phổi'}, {'result': None}, {'result': 'đau đầu, rối loạn thần kinh não'}, {'result': 'mất ngủ, đau nửa đầu'}, {'result': 'đau nửa đầu bên trái hơn 10 ngày'}, {'result': 'choáng, chóng mặt, rối loạn tiền đình'}, {'result': 'động kinh, đau đầu'}]}
Progress saved after batch 1
Processing batch 2, records 10 to 19
Processing 10 records
{'results': [{'result': 'đau đầu, ngủ hay mơ'}, {'result': 'tê bì nửa mặt'}, {'result': 'đau nửa đầu gauche,ù tai gauche'}, {'result': 'bị co cơ,căng dây thần kinh'}, {'result': 'thiếu máu não'}, {'result': 'xơ vữa động mạch'}, {'result': 'đau mắt,tai'}, {'result': 'bại não'}, {'result': 'hoa mắt chóng mặt,nhức đầu cơn trung khu'}, {'result': 'thịnh thoảng,mất ngủ'}]}
Progress saved after batch 2
Processing batch 3, records 20 to 29
Processing 10 records
{'results':

KeyboardInterrupt: 

In [33]:
processed_df.tail()

,partner_id,specialist_id,status,gender,province_id,age,reason_combind,specialist_name,correct_prediction,ids,logs,processed_symptoms
95,4,18,2,NaN,1,24,rối loạn giấc ngủ,thần kinh,NaN,NaN,NaN,rối loạn giấc ngủ
96,3,18,2,NaN,1,61,bị pakison,thần kinh,NaN,NaN,NaN,bệnh parkinson
97,3,18,2,NaN,31,33,"đau đầu kéo dài, ngủ chập chờn",thần kinh,NaN,NaN,NaN,"đau đầu kéo dài, ngủ chấp chân"
98,1,18,2,NaN,30,39,"mệt mỏi, nóng nực trong ngực",thần kinh,NaN,NaN,NaN,"mệt mỏi, nóng nực trong người"
99,3,18,2,NaN,1,46,"hoa mắt, chóng mặt",thần kinh,NaN,NaN,NaN,"hoa mắt, chóng mặt"
